<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/create_dataset/test_data_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import re
import gc
import pickle
import itertools
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime as dt
from pathlib import Path
from tqdm import tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # ut(10 digit) -> date

# pandas settings -----------------------------------------
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.5f}'.format

# Graph drawing -------------------------------------------
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc
from matplotlib_venn import venn2, venn2_circles
from matplotlib import animation as ani
from IPython.display import Image
from pylab import imread
from IPython.display import HTML

plt.rcParams["patch.force_edgecolor"] = True
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

%matplotlib inline
%config InlineBackend.figure_format='retina'

# ML -------------------------------------------
from sklearn.preprocessing import LabelEncoder

def unpickle(filename):
    with open(filename, 'rb') as fo:
        p = pickle.load(fo)
    return p

def to_pickle(filename, obj):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, -1)



class FeatureStore():
    
    # necessayr to re-check
    floor_convert = {'1F' :  0, '2F' : 1, '3F' : 2, '4F' : 3, '5F' : 4, 
                     '6F' : 5, '7F' : 6, '8F' : 7, '9F' : 8,
                     'B'  : -1, 'B1' : -1, 'B2' : -2, 'B3' : -3, 
                     'BF' : -1, 'BM' : -1, 
                     'F1' : 0, 'F2' : 1, 'F3' : 2, 'F4' : 3, 'F5' : 4, 
                     'F6' : 5, 'F7' : 6, 'F8' : 7, 'F9' : 8, 'F10': 9,
                     'L1' : 0, 'L2' : 1, 'L3' : 2, 'L4' : 3, 'L5' : 4, 
                     'L6' : 5, 'L7' : 6, 'L8' : 7, 'L9' : 8, 'L10': 9, 
                     'L11': 10,
                     'G'  : 0, 'LG1': 0, 'LG2': 1, 'LM' : 0, 'M'  : 0, 
                     'P1' : 0, 'P2' : 1,}
    
    df_types = ['accelerometer',
                'accelerometer_uncalibrated',
                'beacon',
                'gyroscope',
                'gyroscope_uncalibrated',
                'magnetic_field',
                'magnetic_field_uncalibrated',
                'rotation_vector',
                'waypoint',
                'wifi']
    
    # https://github.com/location-competition/indoor-location-competition-20
    df_type_cols = {'accelerometer': ["timestamp", "x", "y", "z", "accuracy"],
                'accelerometer_uncalibrated': ["timestamp", "x", "y", "z", 
                                               "x2", "y2", "z2", "accuracy" ],
                'beacon': ["timestamp", "uuid", "major_id", "minor_id", "tx_power", 
                           "rssi", "distance", "mac_addr", "timestamp2"],
                'gyroscope': ["timestamp", "x", "y", "z", "accuracy"],
                'gyroscope_uncalibrated': ["timestamp", "x", "y", "z", 
                                           "x2", "y2", "z2", "accuracy" ],
                'magnetic_field': ["timestamp", "x", "y", "z", "accuracy"],
                'magnetic_field_uncalibrated': ["timestamp", "x", "y", "z", 
                                                "x2", "y2", "z2", "accuracy" ],
                'rotation_vector': ["timestamp", "x", "y", "z", "accuracy"],
                'waypoint': ["timestamp", "x", "y"],
                'wifi': ["timestamp", "ssid", "bssid","rssi","frequency",
                         "last_seen_timestamp",]}

    dtype_dict = {}
    dtype_dict["accelerometer"] = {"timestamp":int, "x":float, "y":float, "z":float, 
                                   "accuracy":int}
    dtype_dict["accelerometer_uncalibrated"] = {"timestamp":int, "x":float, "y":float, 
                                                "z":float, "x2":float, "y2":float, 
                                                "z2":float, "accuracy":int}
    dtype_dict["beacon"] = {"timestamp":int, "uuid":str, "major_id":str, 
                            "minor_id":str, "tx_power":int,  "rssi":int, 
                            "distance":float, "mac_addr":str, "timestamp2":int}
    dtype_dict["gyroscope"] = {"timestamp":int, "x":float, "y":float, "z":float, 
                               "accuracy":int}
    dtype_dict["gyroscope_uncalibrated"] = {"timestamp":int, "x":float, "y":float, 
                                            "z":float, "x2":float, "y2":float, 
                                            "z2":float, "accuracy":int}
    dtype_dict["magnetic_field"] = {"timestamp":int, "x":float, "y":float, 
                                    "z":float, "accuracy":int}
    dtype_dict["magnetic_field_uncalibrated"] = {"timestamp":int, "x":float, 
                                                 "y":float, "z":float, "x2":float, 
                                                 "y2":float, "z2":float, "accuracy":int}
    dtype_dict["rotation_vector"] = {"timestamp":int, "x":float, "y":float, 
                                     "z":float, "accuracy":int}
    dtype_dict["waypoint"] = {"timestamp":int, "x":float, "y":float, "z":float}
    dtype_dict["wifi"] = {"timestamp":int, "ssid":str, "bssid":str,
                          "rssi":int,"frequency":int, "last_seen_timestamp":int}

    def __init__(self, site_id, floor, path_id, 
                 input_path="../input/indoor-location-navigation/",
                 save_path="../mid"):
        self.site_id = site_id.strip()
        self.floor = floor.strip()
        self.n_floor = self.floor_convert[self.floor]
        self.path_id = path_id.strip()
        
        self.input_path = input_path
        assert Path(input_path).exists(), f"input_path do not exist: {input_path}"
        
        self.save_path = save_path
        Path(save_path).mkdir(parents=True, exist_ok=True)
        
        self.site_info = SiteInfo(site_id=self.site_id, floor=self.floor, input_path=self.input_path)
        
    def _flatten(self, l):
        return list(itertools.chain.from_iterable(l))
    
    def multi_line_spliter(self, s):
        matches = re.finditer("TYPE_", s)
        matches_positions = [match.start() for match in matches]
        split_idx = [0] + [matches_positions[i]-14 for i in range(1, len(matches_positions))] + [len(s)]
        return [s[split_idx[i]:split_idx[i+1]] for i in range(len(split_idx)-1)]
    
    def load_df(self, ):
        path = str(Path(self.input_path)/f"train/{self.site_id}/{self.floor}/{self.path_id}.txt")
        with open(path) as f:
            data = f.readlines()
        
        modified_data = []
        for s in data:
            if s.count("TYPE_")>1:
                lines = self.multi_line_spliter(s)
                modified_data.extend(lines)
            else:
                modified_data.append(s)
        del data
        self.meta_info_len = len([d for d in modified_data if d[0]=="#"])
        self.meta_info_df = pd.DataFrame([m.replace("\n", "").split(":") 
                                          for m in self._flatten([d.split("\t") 
                                                                  for d in modified_data if d[0]=="#"]) if m!="#"])

        data_df = pd.DataFrame([d.replace("\n", "").split("\t") for d in modified_data if d[0]!="#"])
        for dt in self.df_types:
            # select data type
            df_s = data_df[data_df[1]==f"TYPE_{dt.upper()}"]
            if len(df_s)==0:
                setattr(self, dt, pd.DataFrame(columns=self.df_type_cols[dt]))
            else:
                # remove empty cols
                na_info = df_s.isna().sum(axis=0) == len(df_s)
                df_s = df_s[[i for i in na_info[na_info==False].index if i!=1]].reset_index(drop=True)
                
                if len(df_s.columns)!=len(self.df_type_cols[dt]):
                    df_s.columns = self.df_type_cols[dt][:len(df_s.columns)]
                else:
                    df_s.columns = self.df_type_cols[dt]
            
                # set dtype          
                for c in df_s.columns:
                    df_s[c] = df_s[c].astype(self.dtype_dict[dt][c])
                                     
                # set DataFrame to attr
                setattr(self, dt, df_s)
    
    def get_site_info(self, keep_raw=False):
        self.site_info.get_site_info(keep_raw=keep_raw)
            
    def load_all_data(self, keep_raw=False):     
        self.load_df()
        self.get_site_info(keep_raw=keep_raw)
        
    def __getitem__(self, item):
        if item in self.df_types:
            return getattr(self, item)
        elif item=="sensors":
            try:
                return getattr(self, "sensor_df")
            except:
                self.sensor_df = pd.concat([feature["magnetic_field"].set_index("timestamp"), 
                                       feature["accelerometer"].set_index("timestamp"), 
                                       feature["gyroscope"].set_index("timestamp")], axis=1)
                if self.sensor_df.shape[1]==12:
                    self.sensor_df.columns = ["mag_x", "mag_y", "mag_z", "mag_acc", 
                                              "acc_x", "acc_y", "acc_z", "acc_acc",
                                              "gyr_x", "gyr_y", "gyr_z", "gyr_acc", ]
                else:
                    self.sensor_df.columns = ["mag_x", "mag_y", "mag_z", 
                                              "acc_x", "acc_y", "acc_z", 
                                              "gyr_x", "gyr_y", "gyr_z",  ]
                return self.sensor_df
        else:
            return None
    
    def save(self, ):
        # to be implemented
        pass
    
    
class SiteInfo():
    def __init__(self, site_id, floor, input_path="../input/indoor-location-navigation/"):
        self.site_id = site_id
        self.floor = floor
        self.input_path = input_path
        assert Path(input_path).exists(), f"input_path do not exist: {input_path}"
        
    def get_site_info(self, keep_raw=False):
        floor_info_path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/floor_info.json"
        with open(floor_info_path, "r") as f:
            self.floor_info = json.loads(f.read())
            self.site_height = self.floor_info["map_info"]["height"]
            self.site_width = self.floor_info["map_info"]["width"]
            if not keep_raw:
                del self.floor_info
            
        geojson_map_path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/geojson_map.json"
        with open(geojson_map_path, "r") as f:
            self.geojson_map = json.loads(f.read())
            self.map_type = self.geojson_map["type"]
            self.features = self.geojson_map["features"]
            
            self.floor_coordinates = self.features[0]["geometry"]["coordinates"]
            self.store_coordinates = [self.features[i]["geometry"]["coordinates"] 
                                          for i in range(1, len(self.features))]
                
            if not keep_raw:
                del self.geojson_map
    
    def show_site_image(self, ax):
        path = f"{self.input_path}/metadata/{self.site_id}/{self.floor}/floor_image.png"
        ax.imshow(imread(path), extent=[0, self.site_width, 0, self.site_height])

    def draw_polygon(self, size=8, only_floor=False):

        fig = plt.figure()
        ax = plt.subplot(111)
            
        xmax, xmin, ymax, ymin = self._draw(self.floor_coordinates, ax, calc_minmax=True)
        if not only_floor:
            self._draw(self.store_coordinates, ax, fill=True)
        plt.legend([])
        
        xrange = xmax - xmin
        yrange = ymax - ymin
        ratio = yrange / xrange
        
        self.x_size = size
        self.y_size = size*ratio

        fig.set_figwidth(size)
        fig.set_figheight(size*ratio)
        # plt.show()
        return ax
        
    def _draw(self, coordinates, ax, fill=False, calc_minmax=False):
        xmax, ymax = -np.inf, -np.inf
        xmin, ymin = np.inf, np.inf
        for i in range(len(coordinates)):
            ndim = np.ndim(coordinates[i])
            if ndim==2:
                corrd_df = pd.DataFrame(coordinates[i])
                if fill:
                    ax.fill(corrd_df[0], corrd_df[1], alpha=0.7)
                else:
                    corrd_df.plot.line(x=0, y=1, style="-", ax=ax)
                        
                if calc_minmax:
                    xmax = max(xmax, corrd_df[0].max())
                    xmin = min(xmin, corrd_df[0].min())

                    ymax = max(ymax, corrd_df[1].max())
                    ymin = min(ymin, corrd_df[1].min())
            elif ndim==3:
                for j in range(len(coordinates[i])):
                    corrd_df = pd.DataFrame(coordinates[i][j])
                    if fill:
                        ax.fill(corrd_df[0], corrd_df[1], alpha=0.6)
                    else:
                        corrd_df.plot.line(x=0, y=1, style="-", ax=ax)
                        
                    if calc_minmax:
                        xmax = max(xmax, corrd_df[0].max())
                        xmin = min(xmin, corrd_df[0].min())

                        ymax = max(ymax, corrd_df[1].max())
                        ymin = min(ymin, corrd_df[1].min())
            else:
                assert False, f"ndim of coordinates should be 2 or 3: {ndim}"
        if calc_minmax:
            return xmax, xmin, ymax, ymin
        else:
            return None
        

In [ ]:
from matplotlib import animation as ani
import functools
from IPython.display import Image
plt.rcParams["font.size"] = 7

def animate(axes, nframe):
    global num_frame
    
    ts = feature.waypoint.timestamp.values[nframe]
    
    # axes[0]------------------------------
    wp =feature.waypoint.iloc[nframe]
    axes[0].text(x=wp.x, y=wp.y, s=f"{nframe}", fontsize=6)
    
    # axes[1]------------------------------
    axes[1].vlines(ts, -3.5, 3.5, "k", lw=1)
    axes[1].text(ts, -1, f"{nframe}", fontsize=8)
    
    # axes[2]------------------------------
    axes[2].vlines(ts, -40, 40, "k", lw=1)
    axes[2].text(ts, -20, f"{nframe}", fontsize=8)


def drow_path_gyr_mag(feature, gif_name):    
    ts_min = feature.waypoint.timestamp.min()
    ts_max = feature.waypoint.timestamp.max()
    query = f"{ts_min} <= timestamp and timestamp <= {ts_max}"
    sensor_df = feature["sensors"].reset_index().query(query)
    sensor_df["ts_diff"] = sensor_df.timestamp.diff() / 1000
    sensor_df["x_pos"] = 0
    sensor_df["y_pos"] = 0

    sensor_df["gyr_z_dif_ts"] = sensor_df["gyr_z"] * sensor_df["ts_diff"].fillna(0.02)
    fig, axes = plt.subplots(3, 1,
        gridspec_kw=dict(width_ratios=[1], 
                         height_ratios=[4,1,1], 
                         wspace=0.1, 
                         hspace=0.3),)
    # axes[0]------------------------------
    feature.site_info.show_site_image(ax=axes[0])
    axes[0].grid(False)
    axes[0].tick_params(labelsize=7)
    # axes[1]------------------------------
    gyro_data = sensor_df.set_index("timestamp").gyr_z_dif_ts.cumsum()
    gyro_data.plot(lw=1, label="gyr_z_dif_ts", ax=axes[1])
    axes[1].hlines(np.pi, ts_min, ts_max, "gray", linestyle="--", lw=0.5)
    axes[1].hlines(-np.pi, ts_min, ts_max, "gray", linestyle="--", lw=0.5)
    axes[1].legend(loc="best", fontsize=8)
    axes[1].set_title("gyr_z", fontsize=8)
    axes[1].set_xlabel('timestamp', fontsize=8)
    axes[1].tick_params(labelsize=7)
    # axes[2]------------------------------
    sensor_df.set_index("timestamp").mag_x.plot(lw=1, label="mag_x", ax=axes[2])
    sensor_df.set_index("timestamp").mag_y.plot(lw=1, label="mag_y", ax=axes[2])
    axes[2].legend(loc="best", fontsize=8)
    axes[2].set_title("magnetic_field_x, y", fontsize=8)
    axes[2].set_xlabel('timestamp', fontsize=8)
    axes[2].tick_params(labelsize=7)
    
    animate_ = functools.partial(animate, axes)
    anim = ani.FuncAnimation(fig, animate_, frames=len(feature.waypoint))

    fig.set_figwidth(8)
    fig.set_figheight(12)
    anim.save(gif_name, fps=2, dpi=256)
    plt.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_meta = glob("/content/drive/MyDrive/use_data/test_/*")
test_meta_org = pd.DataFrame(test_meta)
test_meta = pd.DataFrame(test_meta_org[0].str.split("/", expand=True)[6])
test_meta.columns = ["test_id"]
test_meta["test_id"] = test_meta["test_id"].str.replace(".txt", "")
test_meta["path"] = test_meta_org[0]
test_meta.head()

In [ ]:
for index, columns in test_meta.iterrows():
    if columns[0] in ['046cfa46be49fc10834815c6', '05d052dde78384b0c543d89c',
                        '0c06cc9f21d172618d74c6c8', '146035943a1482883ed98570',
                        '1ef2771dfea25d508142ba06', '3506b3b626f494b0f0b934ca',
                        '3e0aebb66ef39150bbc27c24', '3e1d46017fbfcc8136bd1e9b',
                        '412da1891c4780f6f0f7f4bc', '6d89334316127640cff99800',
                        '72963a8c7eb520c56f88a536', '7d2d723a30cce824aad9915b',
                        '947e17f82dbddfbdb4cb2447', '986924433ab01afa81a59e7a',
                        'a3c5aadf824a220327e9cdc6', 'ac1a2e8f11ab64d729199969',
                        'bb84ab5e77fc9f5fdbd52827', 'bd0f2c8626679895f3338ca5',
                        'bd5921cd1008382e2f537b53', 'ce80c08d743f15e6586741f6',
                        'd592885af4e6e380c376dc55', 'd6e09245089299ec8e5173e7',
                        'dd4cbd69218f610f27cf33c8', 'de9cef0d69383e47bd74a64c',
                        'e91afaba9603500e8e28a454', 'ea777463a91a1293fc79fc52',
                        'ec9aee8ee72b1902bd83edf5', 'ed77f28aeaf89b317bc380fa',
                        'ffcd9524c80c0fa5bb859eaf']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5a0546857ecc773753327266')
    elif columns[0] in ['627d97e6be0e2f1f630bdb37', '7bc4213a6a76c5cb03de2d69', 'b1b61e500f46136f8ee8e94f']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5c3c44b80379370013e0fd2b')

    elif columns[0] in ['108d983b84def2bd48bdca36', '58a7b7d31f5e8898403d1607',
 '76f7af4eeffffde457d018bb', '9cc4412ff73ec37e30be9d9f',
 'bfcb651b80df271b79d344dd']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5d27075f03f801723c2e360f')

    elif columns[0] in ['0884650b5640ded8c63750fc', '0d4d75060f346ce93d416d01',
 '0eeb9f6b6d4c8f39fb34b577', '13d970ab11b767d7ae3e4ee6',
 '14f45baa63b4d3a700126af6', '15a04a78926385821aa96a62',
 '1747b3e80ba80910158c0cef', '17540fdca1dd0b3d5620336d',
 '19bf3d9c5236d0f7429baa6a', '1b9fb13726d1030612228aeb',
 '238a279ef92528dd92aeb2e3', '23b4c8eb4b41d75946285461',
 '25a8bb2034f511717c46e536', '2db317d8880522bb060d0819',
 '2f33560940ba465c8226c951', '30ed8393bd1d301cd380c06b',
 '3dbf5c79cfcc18eaca1a7d3b', '3e6892787428117fb4b061d7',
 '3f815bfeee94086f60be0d09', '3fc0e68775cde4a5fce6cf2d',
 '4339bf75463dd4e25aa84758', '43b5cb4125a03434d3ee065d',
 '4b3c36b97204adfe690ccb00', '4d4315146c8acfc8f7a20550',
 '4d51d8913e1c785e0a1dddef', '4dcb3e9d10defbea8e8007c2',
 '4e2aedb331c566b0e7684ffc', '50b0a8d77c8c34368ea4bfc7',
 '50f24c8ed539bdc44ebf1dd8', '5258ce6a8b63b1f30d04a2d5',
 '59cf31f3a5cd8a048a3167d5', '5f58ff915fb692e2507cfdd5',
 '60e0899d24db2cbaa927abcd', '673bee9ae2df0a2d44d178ec',
 '689ac374ec67d0f0b0fe4b0a', '6913609ad03d8092c8b5b822',
 '6b61e3eb48c15f554f18de21', '7241824cc3075197e0ded020',
 '760075793d3822dd8bf1bf31', '7e1432b1a371d5d3e8ae34d3',
 '8304231d833d5427ff4f4da1', '833d0a5609a088d7c15d2031',
 '8545d8808a11f6b48d112310', '949b5f7dd5469b2b7c4d57b4',
 '9b97a70dcbae27c225bd36f9', 'a7e5d323e7ba3df29a4bb918',
 'aec3b618b191da96f0eb2c10', 'c06224bdefec4d102f001601',
 'c61792dc8a5208a73bfea103', 'c7c9d2f5113d91afa2b7e94d',
 'ca363c1b30692cf0598bea76', 'cc23760c1abbefee0791b1f2',
 'd148573a2b01245f06e1de50', 'd7c936e862946ca5f6926512',
 'd905d9e3a421b0ad31a8a7c0', 'dc641944142a65f5ddf845bb',
 'decc9a540ad050e541a897c3', 'ed7d88f09de7395a7de74ce4',
 'f043e6ddf42667290fa8b239', 'f31f1f8ec42e10d23cb54ec4']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5d27096c03f801723c31e5e0')

    elif columns[0] in ['18ab7e85351d03d1cd691a73', '2fed60cb00f71d86ef78b5b4',
 '4fbd93217986b45372ebedd4', '500913fc93d997b6803dd6df',
 '698dc1d1a1885908e8fbfe4c', '7727672abec7d70216173223',
 '9dedc3819b9ac8444b1f30f4', 'a25f020b7571d262ff5952ed',
 'a606cb39b74b8e0d45c8c8b8', 'b760d372b55d88f398110f48',
 'b7deb644d3ec40fb81cad6e2', 'bb73ec8779aaa0509bea454a',
 'bf5543efb9497b2dd44ca7da', 'cf505e00f50df4548ef98d05',
 'd712536c776ed08ddfcb73c6', 'e8c5c09da1a03d6a980ebf8c',
 'ea6f61a237dc4e79b251422a']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5d27097f03f801723c320d97')

    elif columns[0] in ['7015c5709467b8fdb974daf4', '81badbb3aded20c36ae22bc2',
 'b8d7ddb54d74a9450d7eb17c', 'bbe172d896a38bc15fca3062',
 'c9f3dcee23725759730eaf32']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5d27099f03f801723c32511d')

    elif columns[0] in ['0412d582bb8a2c89400a1ffb', '236bb7800311ca69deeb0a16',
 '41451a862cca51c9df6bf152', '5582270fcaee1f580de9006f',
 '60d4eac93ce6283cdc6e11b6', '6913691fa6371ba1be81276e',
 '897457315805f1d5b209c692', '89f1ea2d1e3a876af40dadac',
 '8ab34318f66dffa710ec53cd', '8d64c2a46bc9b91426969ef6',
 '9217871066df2966c942b2eb', '987dedd74979b11fe4f06987',
 '98942ecbc18dc774835563aa', '9950fd1962191dfe62c8a366',
 'd3db994677fd0e0ed62339d2', 'e1539744b6f7668a040ac968',
 'e9db664dfc3a965581a24972', 'f1571d150cf1aafde30fd79a',
 'f8ebbce91c3aa8870faba81e', 'f971e55da42b78e4d7a628e8']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5d2709a003f801723c3251bf')

    elif columns[0] in ['04b259d70f2b503f2af14c35', '26c2e2bc3d0f36109337b502',
 '2bc46d57dc3f3ba0480e9f08', '2bf1ea3f5895fbdadd5b4a74',
 '346bf8f7f1c62bcb4cd6e307', '3c972e7cbba71d1c684dd34f',
 '44bc288dc5e6d7b2819f88c5', '472be94f5be907c04c932114',
 '4affd104e0ec7a7806edfa78', '4ff5b9152353b3dc458992fa',
 '5727f10c5188b327a5a6a93e', '5ca85be3a9fc9cd53ea8c7f1',
 '6ea1015ef1f27bbd15192eb6', '6fe7c294e3a47e17aeb240b2',
 '796c8d48da4b59d8c47d2c3a', '79dbd47db3541d41acf3645c',
 '86a46c7bf9fe29a67980b1a7', '8817286b357023b71bd0b696',
 '9690a79f77581d8c60302d2c', 'ad228ceb56e37f3416a910a3',
 'b2c1f3b0b84bae349a85a76f', 'c50cc8ab7c5008f50a38e70d',
 'c70272750cc48acef9827dcb', 'e39b941dd2bcc2bffdc433b1',
 'e492f8f51c101a523a137755', 'e52fbc1a0b00f58b565066c9',
 'ec9f3bbadfdd3464ed3ddf28', 'ee89473de9b2774fd7d8f029',
 'eeb5b1d0e23c0675e095b98a', 'eef7be7d57a15ecb1ab3c578',
 'ff141af01177f34e9caa7a12']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5d2709b303f801723c327472')

    elif columns[0] in ['05a6d4cdf3d1eb9b5671f71b' '07fa2d05e9b5a9841b7bfae3',
 '2116a2a90f494f7773b550ef', '2379a535c2221d54e8caf6ff',
 '35b801b06a7816dc8a77080c', '3aac7e63e395fe2902ce9bd9',
 '3ca3e65a5fc44603f0c827ba', '4283389c88821c3b31d07df5',
 '505a039e760d4875c9c2d77f', '5853ed01a28b1d938e25b2d7',
 '6085d88383432829bafe3147', '672b4e3c9b09a218a4e74974',
 '71d0d1f8c3abeb9d6de37c88', '7276fac9b55aa417322e20f8',
 '854dedd673b748ac7a14c6a7', '8c7c75845cfba8c2b874d2ea',
 '99ea88fcd02f1d7cb4d141bf', '9b720b32ec8fac3b82f19ae1',
 'a1a53ec2c1992d3947dd4480', 'a403d556f6ebc1343e0b27ee',
 'a526dbdf164aa0a28f510c49', 'a5e437e7708788a70e8bc8ec',
 'aa49b4006c18d1d2a28cd049', 'b308a8c4ffeee07f455ffb19',
 'b51a662297b90657f0b03b44', 'b6f2d87525cec0dcd65b693c',
 'bb8d876705a0d8b453791035', 'c83dfe151e943e78333cec13',
 'cfda785b67b4b17c58b911b0', 'd2a8a8bd9569c33bc43ec14e',
 'd72ceac4628436a540910d98', 'dcd4bfc49601637d9339808d',
 'e9816721ed502a5414ee6aa4', 'f5fa36c503893adeb5294d52']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5d2709bb03f801723c32852c')

    elif columns[0] in ['08a29ac71fe460a44ca73b38', '1892ff54553115169341d72e',
 '19ea02c9635f48c2bc7a161f', '1a2cc4c5793fa089dd58ddd1',
 '227b32473aa440c6497e2ae9', '2ae8bd41bbf2eb78d76d737d',
 '2e343fa6a9237d61d0f99ccc', '36e11c56d336fd708d34ac37',
 '3c612b99b0f6800992eacf43', '46c865d487fa011022507710',
 '60bc1c7de6f805c28228b3fe', '659307d58a1bd977d58bc94f',
 '728aef201c2b68acc370b7f8', '78790e6e6a7e658c5e25955f',
 '7a9ed75a485605db82f143d5', '9342f71811c45c856bf964c7',
 '9432b20a613c4c0a40993531', '967b41cea41974071e36f31d',
 '9e014c6bd4f50e1256624ae5', 'a2627e7d41e7e1c686bb74f9',
 'a55eb4ea2b0556f96143f176', 'ae4af47ef601ee8664a18778',
 'c1bee305c2d4212c18c91e4e', 'cb9399bab0ead661258086ae',
 'cebe97fd2e7949b281656193', 'dc5f905014579b7fb857d2f0',
 'de9160a4b1db66a453ebfb68', 'dffd0a427bc76233ef658a1d',
 'fa4d3f7e66fdb499255ddaf8', 'fbfbf091d226818a4fc3516c',
 'fca68c2a470d254cece58870']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5d2709c303f801723c3299ee')

    elif columns[0] in ['06882da3694b7160c0f105f5', '07db4eec1fc8df5040d86602',
 '0a5c442ffa2664e89e18262d', '0df4917338687fdbc93448f5',
 '0fee6e9fd162d281b15c11df', '167fd824f8c93490e4714032',
 '19e4a759bff01332b4216bda', '1e85728385bc83e6bb650798',
 '2a83a489565dfa1d77af9f51', '2b4bacedc942ffcb523ff20f',
 '2b9462352bb399a34091d861', '2c5da83393691efba7f8da42',
 '2d17cc2d5c1660c51e00d505', '2de4f5fea373eb7b69a84824',
 '38876a9e9fee20526c6890bf', '44bb4250c66f06bffc554828',
 '4573afa8da16a43e0e2a355f', '4d497d31be6d37d79186717e',
 '507749a1187be5b582671c62', '50dfc9af184b9f9e05ef1ae2',
 '52ad8c760ff9978d0949deed', '5f0252bc39ee6688384856d4',
 '610f2c07b26508790d1cd355', '71675faaf4dedd23ce8e2c39',
 '7316acadafc00dbfdb47ca10', '7c19987b64849e60e831c394',
 '7d401b038d6fdb08f4f8197d', '8c1fdb5fa1bafa5b56ecc4e0',
 '8c4e95647f1f1277f434babc', '8d348ecb29022e18299be281',
 '9609fb2f1c62cf3b7ac08521', '9fd3fbeb694c4b7a37c125d5',
 'a0c05d232b01247a9ba48be0', 'a221eeef88b64a9e58c95f6f',
 'a4609e6c40ba7c527c8c5da2', 'b1f1e78acfb77ad82d538183',
 'b2dd56b88cf0d00902503eba', 'b7987955dad0b6685500132b',
 'c49c72f7f7036cc8a661917e', 'd6daf4cca1df91ce7f5ca2c6',
 'd817eb4bd2f39280ea1037b9', 'd95450e3be51a1d7d284a29b',
 'db7b0850aed5577702f151c4', 'eafbdd565c2e1e7650607e43',
 'eb1988b0b15be78203297cbf', 'f0d402a465655bfc8ab41242',
 'f4bd7479dbe91f3d18f44c49', 'f5868ddf1e5f4ccf9a648517',
 'f64fb235136206764a4c23fc', 'fc99dca042a1ed67dc80a0fe',
 'fdac8be560a05943aba7e41f']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5d2709d403f801723c32bd39')

    elif columns[0] in ['16e7537d3cb1b1b206cf08a8', '21459c30ba2ac052fed92f25',
 '219d400e61f8e06a066572f8', '2ff4941b091ecb02e41997cd',
 '32e88857b1938abaebce9a76', '36481f178c4ce333acccd58e',
 '3843f1ee3f43653993d60447', '3bc2ad10daefe7fc6e57bdca',
 '4209447de426d0c86761703a', '4e7665b783a3113c7e73cb94',
 '4f66cef6a69aaf88468f7ab7', '54b4ec188d91140a6a2ee030',
 '5f4f516acba921593e55cc4b', '633b9baf7b8415ee819c26a7',
 '6cef04795d0f3a4313958f16', '6f705385cf0cebca4b0ca6e7',
 '7caee2ed97f52218042a662f', '8f134b651d254092c94e855f',
 '9dd916c57ffba275b1baa323', 'a7bb61478b112c3eedcbb6ed',
 'a9cc261267276e486328a30d', 'ad733a1791d3fdd21532677b',
 'b433961afd6c429fc2dd0971', 'b8b2f59af006df03c9d3e0d6',
 'c9a69eab2313afa2086a7927', 'cb278289e8158f129e31cfcd',
 'e3455e53350336857caf08f0', 'e7574977ef4bd5f7a8bd9751',
 'e83cee96a0e4c29297699594', 'f7e3f9a90a270fed8f32b3b7',
 'f8ca6722d390d090d4974316']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5d2709e003f801723c32d896')

    elif columns[0] in ['049bb468e7e166e9d6370002', '085e9207300765e045a942bf',
 '4df1cce533e2aaf6882fb638', '6751305422c4e62ee256a6f0',
 '8584d22e99dc86f336173b15', 'b36d5f0beec291379d2181b6',
 'db152e591fe88b82b706da8d', 'e8bf0626e27589d807a9751d']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5da138274db8ce0c98bbd3d2')

    elif columns[0] in ['0fb75bf676e1cddace3590d5', '15d9cd90a0c1d8c0b2e060f8',
 '2edd1ed9897871186b126a68', '310990c10f799a9b21cd4ff3',
 '78beda79f100af67d5fd91e6', '862a4ac32755d252c6948424',
 '876cd27fc8f63a2800fc3de8', '8ca7fe022118929860699a2f',
 '9e001583893cabee64bf69df', 'b03a2291b48aa904314717db',
 'c816924dba6739d9c63021f7']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5da1382d4db8ce0c98bbe92e')

    elif columns[0] in ['0969b6db91bb1697f99a8a21', '0d08327383a1f146a26a2a2b',
 '136d1cb5b01538b9979aedbe', '1c93cbfe9be665edd768737d',
 '20073e2d139310a2bae877c4', '2fe9bdaed4fb3e1368e07ccc',
 '330b81185e5f5c5513ae7d2f', '41fb71519f03dabae411b661',
 '49ef0e8c42428f0c67c3ed4b', '52265eb3084b4e3f7b08f6ae',
 '5340e354629f97c9348270c0', '6ae5d7dfba87555bb7bd70e6',
 '83ae65e7f2059dae6c015ab5', '97c46c1fc7ebc5930af75743',
 'b1181f6f7f7f4bb4ec48e5c4', 'f1d0b39dc36699ffe5d68e4d',
 'fc3a6efa90c6712df8739fc1']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5da138314db8ce0c98bbf3a0')

    elif columns[0] in ['29781f56d67d950acd783d29', '438a576003e0eef20daf9c35',
 '5b67ae395926dc10ea314317', '75703a1e62aa93e8c56de7aa',
 '7d3f0c002431353bc0a54eeb', '81532abcb8b4e11eec4a8202',
 '90e36817a15d1303999b17c9', 'a11615477afe31e7a204287f',
 'cbe53b25f9fd9c9acea1e6d1', 'd3402272ac97751e0e87cdf3']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5da138364db8ce0c98bc00f1')

    elif columns[0] in ['04029880763600640a0cf42c', '14669681607aa88e65b0d927',
 '242c6ee1e520eff5ef06a45d', '2658f7f85981f264dd9657f0',
 '2670249c99fdb6bba41a8511', '35b82be00f77fc7f1d241575',
 '3a893f2b18e85ef08567debf', '3edd7d06006a5bfc1619fc0e',
 '504b8655852f837f2aca36a7', '5b9d1b98227340deb1d6f185',
 '5e5ee82cd8753120756c78e4', '6c9481054d611cffe044b807',
 '7264eeec0a8d8aa61e51cd5f', '77cabd9a8eaaa49beb66a62b',
 '7814e8ec13281ccf89baedc0', '8a5e440d8d110c0a3539ee9b',
 'a23ca542020c3376f8da1fd9', 'a25a8d841f9dcb508c5ee363',
 'aed8d49bbf0857e88e1d26b5', 'bb938cd9fec6ef383e40f4c0',
 'c3913710fd39499173a055e7', 'e1c113233943f6191b64441c',
 'e3d5b272b2c71179e501519e', 'ee9be1ad62ff5b8c6a44dc3a',
 'f8a2489aa99b08ffddc115ec', 'fc01ec821b1c8cf910b4b895']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5da1383b4db8ce0c98bc11ab')

    elif columns[0] in ['10f9407023d1e00bbd7274f2', '309c38eba30007e666cf7fd0',
 '40268946c8060c91109a58b7', '40ded46d0cb2e060306b26bb',
 '47b83323569055e31d15a469', '4ff6a5f94782360efff2c334',
 '667e1fd9da2bfe14aafdec4b', '679845ce7fb4d5b56d6887c1',
 '6b598c55272851a4185d23aa', '70123d9366c5676db1d1d0dd',
 '74e1f3f41374ba181468248d', '82683559aebe665bb0eebe8e',
 '852fb29bb4996d022496edcc', '85a41fe1d8c529c033810499',
 '89f92d1370b844b8e7644ee8', '8febc00382b315a208c3d9eb',
 '9d89ba9627651adaa7fbf78f', 'aa3bb82db66286095d941035',
 'afd41e65cb00646768654b9c', 'c26b88f19f04117b7e4dd747',
 'cd7cbeadf3cd476110f10c14', 'eb50a89699cb96502bcbb033',
 'f95509fd9560e94d6b4d9b2d']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5da138754db8ce0c98bca82f')

    elif columns[0] in ['030b3d94de8acae7c936563d', '0ab1b38a35a76b0e5bf2ca8d',
 '0b6e7a34b2a1b99280b5d3ca', '0dbf956c380feddc590348e0',
 '1c99e4c566466c0d82fc2d6c', '2148b02cde5342187216fa75',
 '291ca2c9bd7d29242b483157', '2bb6f7e32d3191326d674809',
 '2bdf91b83e265e197ac7b117', '34bb5da757c9c83b42e5ae63',
 '4690d48f649624bdf8a09b1b', '4b8d3aff14914537854587c1',
 '4efb8bc8c17cfc5d9cd71c7f', '5ab7fdd5629656e684c0435a',
 '5df63c6ddcddaa958e520ea5', '66dabc484153c8102481a8e4',
 '819494b56fdb3e65ef378897', '89500baf14748fe9b3a82748',
 '8edbfe4ea14f5b8fe8f47f15', '8ffb53cd60edc17b04e25a45',
 '9bed5343c25574e44914ef21', 'b4188fc441162ab5593a7b45',
 'b82e65dde03a9ff7b676fd37', 'c7ace6aca412926c8c25155e',
 'c93b48c40c3f32e200619f99', 'cc8cd263619bc078e1d10ebf',
 'ce09e807fef6de6b45709d9f', 'cf5cc112ba8a1bbb6f8d93e8',
 'd0be93dbc89e6791ec20afc9', 'dc616633d076e209c093f8ac',
 'dc78498813d06eff01a96320', 'dce5ec866b612e86ec04ac06',
 'e05290eced3105a1b99ee13a', 'ed240a8a5738be91c3743e7f',
 'ed2caa67ce313a4baf428ace', 'f973ee415265be4addc457b1']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5da138764db8ce0c98bcaa46')

    elif columns[0] in ['01c41f1aeba5c48c2c4dd568', '055255f16b549ed22893c0c3',
 '0ce9bd5bf8f2351bfd217774', '240060212e4ee62beb701c5c',
 '2402c2005e29469cbfbf410b', '340f3ca472a404fbef58f8b1',
 '39537bf344b43f10a20e060f', '39a5e60107bdf4c530130c52',
 '3e1ba2182d077b7fd17d524f', '3e61e72b06bc6f787d9b176f',
 '4aaa69d3ff7e13ed5e824aa2', '4c93f4e157d7e283547f1153',
 '536134a5924bfa7840e315de', '5a0cd5dec226a90c3818ec03',
 '6b58ff42c73427fdefe9d2bb', '7efc4326b753c9133e0b908e',
 '965778109ccdb3eb34af8bf0', 'a1a32c28854aeef857ef79a5',
 'a57f8728343b9cb2491e6c86', 'aa8383281d2c90c5576bb4e7',
 'ac93c263f13fb5eda552ca97', 'b40da1395a4ea396026ea3fa',
 'b781698c482098426072d432', 'cad1ea3fa0502ba1bcbebd6d',
 'cad7c5c53d62719991fc4c07', 'dbc3537c9913889864ebe0c1',
 'dbc35fa5280de4c5d553d23a', 'dc7aab767a608ca73ea2e33b',
 'f6228876966d7dae2c39b8ca']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5da138b74db8ce0c98bd4774')

    elif columns[0] in ['053526f9012ca715313120cd', '06832e9b9ca24ff7fe906aee',
 '13ed7262c2b6088cd47de5ed', '1ab6361c3c5a94fc99e7ae68',
 '244c21fb418ea7e21425abde', '24d83dcc0488f735862e4d3d',
 '2ad9c59e934facf0b0245af3', '32bcee0203f7d0cac3ed504b',
 '36c6dd1ccc482e94fa21830c', '3c0ebe929de57c5b1fc65d44',
 '3c56892f6cd0af56fe6b92e0', '48dbda4179aaef31ed33c6e4',
 '491066b9356a9f968e04b09b', '5697db90a6c6bcb3e838a2b9',
 '58279d6ab8c2213722f2ef6b', '5e4f084a93b862d00688673d',
 '644c8ec6504fe83b80677866', '66744e8a5c65a748497cd7c0',
 '6b6d91a5b97ed3c40c1411a7', '723900eb4d292e35981c3012',
 '76fa8b5e46e6dacbdc4f4f39', '791c96c9a472e89ec06e8a85',
 '7be7b7833488818f562d24aa', '7d42a67573f38d52bea01679',
 '861c09b1c67351e92708f91f', '89ab0fc4e49d892ff0156dc7',
 '8c5a946a9eeb739c2a6630ff', '8e577bbf235d81ce625e363a',
 '9113be4094a68ba42ad9daa8', '950c991710b1054f4cd03133',
 '97aad1488e3b02f180a8ae09', '9a9cf9f796fae7f3f8643a04',
 'a2b5878c25761e552230d412', 'a424b6ad3de377df7c9e8958',
 'a5b8249fca6d41f4ce1d4a02', 'adb909f464fd9284b53dbc87',
 'adda251cb945aa23e455ab48', 'b47cbb269b2ddf36e39572bd',
 'b87667c3dd21d8aa17bb6a74', 'c0f79f21ad5f32c573f98886',
 'c5c86b509c716c7872e16214', 'd0ba1a514581ead3c796490b',
 'd670943e9fa2473914f568d4', 'd6820b1b6728bdb201ab7805',
 'dde666fd35199cb019c8ea62', 'e8ba16dafb39e5cd1b38f813',
 'ea18b971347e2ce438bf4557', 'eb40b68130e8aa8d7dc82c23',
 'efb57307a13583ba06bd099c', 'f6eb59b252c21c171d2ce58c',
 'f75c0093a23e5653b6f6b26c']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5da958dd46f8266d0737457b')
    elif columns[0] in ['0389421238a7e2839701df0f', '110637c78f384710d1b3eba6',
 '1d811352eaf5a17cf918d3dd', '24fca45b67e543a4ad737709',
 '270a66500675e14fb3154c63', '29579192e65ea7826a727c42',
 '2a39636327b1f42f80bb08cd', '36b20f235015f30bcd123c86',
 '39cd26c8eec1b8a14a561b28', '408e9eacfcb439043f4ce593',
 '44167d3f37c95565f14a5ef4', '47a960ef9f70f0b662d9d244',
 '5243b058c6691779fae7da2c', '583b831d065fdb022cca3a0c',
 '60386b96032381bf1c7a33f6', '67bf4b03fc22542bb6d51daf',
 '6883b003fd96496e062c6ee1', '6ab9bfd01e48a4efe981c6b0',
 '6dcb3e845f3cab94ca55a790', '6fda297cb93a859000e72b70',
 '7987213c53131fd5bbacce4f', '8cb1f56b74af85d3dd0efcd9',
 '8cb8f0045242464f50342338', '9d5db7dab58374e88b12d227',
 'a87fa5784fd6e0791efe650a', 'ac83e97550ad3e7396fb618b',
 'ad40d2b657a1b847b11b013f', 'b59c09efbc2ba31f7df1d0d1',
 'bbcf74a04032515b461cb2cc', 'bd27a637d58beacf9a51c292',
 'bd6637f1fa9e0074495caaca', 'c32f4436326c9589169be1dd',
 'c7eaa26e0dcb8d3680fd3c70', 'cf1da59d0470e29891469f3c',
 'cf5c7ff181051df6e88a5787', 'd66110d1c177307120b5deb2',
 'da7678bc04757fa05f1f7992', 'dbc4e9263d1a2efd4b7b89fc',
 'e2183337d2df7034a6c80cf3', 'e6fcb043744edde3b9f0559b',
 'e70ce41f2b7ce9cdb7045972', 'e8ed3f22ef83d9515ce36e8f',
 'ea0f8a9d4c8009cb6ee8f738', 'eb4de03a42f451c11ffec60c',
 'ee1f4765f6d48f2d405b52ba', 'ef20280811313a39e58e3a08',
 'ef848c8ded4aed6c886ceaf8', 'f8d84977825b44af9aa220c4',
 'fbd3c96ffd27e8384d298406', 'fd76cacb0312c4bd96701f09']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5dbc1d84c1eb61796cf7c010')

    elif columns[0] in ['068e4f6926e78ff6d338d2cc', '178d893e0fd1457ad7fdadd6',
 '52dc57274881463626784b21', '536f7b36d89b921511f4fdc1',
 '559b85f5e7c3de56619c8853', '5694e13f4bb0bac39806b5ae',
 '663ad3a7b78652036aa4290d', '6e1c6452fd95c2d3e6785f36',
 '7121b385f2dcd034cae8c49d', '73b8ae34d0038aa6dcdcd858',
 '7603ac27c8a6723a333e9fb8', '7861f68134d801946e3e2795',
 '7a4c5c74672a27ba94eb9b66', '8ccd3fc53ad2e63651de4376',
 '9eab95aa6e3838f3fbccf311', 'a7b032ef5441a07e68f073be',
 'a850f1538a738b74d58cb9bf', 'a8cf094dbb3c22d02d10e21d',
 'ad97a5e58725339a560c381e', 'ae2dc138bd8552bf6c785377',
 'b3ddea2aa88e2015223b75a8', 'b406c5c925f3b64d8972b2c0',
 'bc253f69b5d49855785d94da', 'bf132134be51ec4891e9e16c',
 'bf4fb3d50ba5202800859799', 'c2ae6f835a79f6fd53600449',
 'c385db2d2886056b8c42a9e6', 'c694a61c01738864eacd4d5e',
 'cd695bf43420d73e23df3d5a', 'e5c6641300c3ace88914ec7f',
 'e83a1c294b5d138339149fcb', 'ee40c6e25fe024d5da451909',
 'f9f99869dad1b2021ec7a7e2', 'fd4eee038c0ad7f0179e337e',
 'fd64de8c4a2fc5ebb0e9f412']:
        shutil.copy(f'{columns[1]}', '/content/drive/MyDrive/data_24/test/5dc8cea7659e181adb076a3f')